# End-to-End Web Application Deployment

## Полный туториал: От нуля до production

Этот ноутбук демонстрирует полный цикл развертывания веб-приложения (Frontend + Backend + Database) с использованием всех модулей infra-*:

- **infra-template**: Создание проекта
- **infra-aws**: Развертывание инфраструктуры
- **infra-network**: Настройка сети и VPN
- **infra-secrets**: Управление секретами
- **infra-ci**: CI/CD пайплайн
- **infra-monitoring**: Мониторинг и логирование

### Архитектура приложения

```
┌─────────────────────────────────────────────┐
│  CloudFront (CDN)                           │
└──────────────┬──────────────────────────────┘
               │
┌──────────────▼──────────────────────────────┐
│  S3 Bucket (React Frontend)                 │
└─────────────────────────────────────────────┘
               │
┌──────────────▼──────────────────────────────┐
│  ALB (Application Load Balancer)            │
└──────────────┬──────────────────────────────┘
               │
┌──────────────▼──────────────────────────────┐
│  ECS Fargate (FastAPI Backend)              │
└──────────────┬──────────────────────────────┘
               │
┌──────────────▼──────────────────────────────┐
│  RDS PostgreSQL (Database)                  │
└─────────────────────────────────────────────┘
```

## Шаг 1: Создание проекта из infra-template

Используем infra-template для создания базовой структуры проекта.

In [ ]:
# Клонируем template и создаем новый проект
!git clone https://github.com/v-grand/infra-template.git my-webapp
!cd my-webapp && rm -rf .git
!cd my-webapp && git init

# Настраиваем переменные проекта
import os
os.chdir('my-webapp')

# Создаем terraform.tfvars.example
tfvars_content = """
project_name = "my-webapp"
environment  = "production"
aws_region   = "us-east-1"

# Network configuration
vpc_cidr = "10.0.0.0/16"

# Database configuration
db_instance_class = "db.t3.micro"
db_name          = "webapp_db"

# Application configuration
app_cpu    = 256
app_memory = 512
"""

with open('terraform.tfvars.example', 'w') as f:
    f.write(tfvars_content)

print("✅ Проект создан из template")

## Шаг 2: Настройка сети (infra-network)

Создаем VPC, подсети, и настраиваем Tailscale для безопасного доступа.

In [ ]:
%%writefile modules/network/main.tf

# VPC Module from infra-network
module "vpc" {
  source = "git::https://github.com/v-grand/infra-network.git//modules/vpc"

  project_name = var.project_name
  environment  = var.environment
  vpc_cidr     = var.vpc_cidr

  # Availability Zones
  azs             = ["us-east-1a", "us-east-1b", "us-east-1c"]
  private_subnets = ["10.0.1.0/24", "10.0.2.0/24", "10.0.3.0/24"]
  public_subnets  = ["10.0.101.0/24", "10.0.102.0/24", "10.0.103.0/24"]

  enable_nat_gateway = true
  enable_vpn_gateway = false

  tags = {
    Terraform   = "true"
    Environment = var.environment
  }
}

# Tailscale for secure access
module "tailscale" {
  source = "git::https://github.com/v-grand/infra-network.git//modules/tailscale"

  vpc_id            = module.vpc.vpc_id
  subnet_id         = module.vpc.private_subnets[0]
  tailscale_authkey = var.tailscale_authkey
}

output "vpc_id" {
  value = module.vpc.vpc_id
}

output "private_subnets" {
  value = module.vpc.private_subnets
}

output "public_subnets" {
  value = module.vpc.public_subnets
}

## Шаг 3: Настройка секретов (infra-secrets)

Используем SOPS для шифрования чувствительных данных.

In [ ]:
# Создаем файл с секретами
secrets_content = """
db_password: "CHANGE_ME_STRONG_PASSWORD"
jwt_secret: "CHANGE_ME_JWT_SECRET_KEY"
api_key: "CHANGE_ME_API_KEY"
tailscale_authkey: "tskey-auth-XXXXX"
"""

with open('secrets.yaml', 'w') as f:
    f.write(secrets_content)

# Шифруем с помощью SOPS (требуется настроенный KMS)
!sops --encrypt --kms 'arn:aws:kms:us-east-1:ACCOUNT_ID:key/KEY_ID' secrets.yaml > secrets.enc.yaml

# Удаляем незашифрованный файл
!rm secrets.yaml

# Добавляем в .gitignore
with open('.gitignore', 'a') as f:
    f.write('\nsecrets.yaml\nterraform.tfvars\n*.tfstate\n*.tfstate.backup\n')

print("✅ Секреты зашифрованы и защищены")

## Шаг 4: Развертывание инфраструктуры (infra-aws)

Создаем основную инфраструктуру: RDS, ECS, ALB, S3.

In [ ]:
%%writefile main.tf

terraform {
  required_version = ">= 1.0"
  
  backend "s3" {
    bucket = "my-webapp-terraform-state"
    key    = "production/terraform.tfstate"
    region = "us-east-1"
    encrypt = true
  }
}

provider "aws" {
  region = var.aws_region
}

# Network module (created above)
module "network" {
  source = "./modules/network"
  
  project_name      = var.project_name
  environment       = var.environment
  vpc_cidr          = var.vpc_cidr
  tailscale_authkey = var.tailscale_authkey
}

# RDS PostgreSQL Database
module "database" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/rds"

  identifier        = "${var.project_name}-db"
  engine            = "postgres"
  engine_version    = "14.7"
  instance_class    = var.db_instance_class
  allocated_storage = 20

  db_name  = var.db_name
  username = "admin"
  password = var.db_password

  vpc_id             = module.network.vpc_id
  subnet_ids         = module.network.private_subnets
  
  backup_retention_period = 7
  skip_final_snapshot    = false
}

# ECS Cluster for Backend
module "ecs_cluster" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/ecs"

  cluster_name = "${var.project_name}-cluster"
  
  vpc_id          = module.network.vpc_id
  private_subnets = module.network.private_subnets
  public_subnets  = module.network.public_subnets
}

# S3 Bucket for Frontend
module "frontend_bucket" {
  source = "git::https://github.com/v-grand/infra-aws.git//modules/s3"

  bucket_name = "${var.project_name}-frontend"
  
  website_configuration = {
    index_document = "index.html"
    error_document = "index.html"
  }
  
  enable_versioning = true
}

In [ ]:
# Инициализация и применение Terraform
!terraform init
!terraform plan -out=tfplan

# Применяем изменения (раскомментируйте для реального развертывания)
# !terraform apply tfplan

print("✅ Инфраструктура готова к развертыванию")

## Шаг 5: Настройка CI/CD (infra-ci)

Создаем GitHub Actions workflow для автоматического развертывания.

In [ ]:
%%writefile .github/workflows/deploy.yml

name: Deploy Web Application

on:
  push:
    branches: [main]
  pull_request:
    branches: [main]

jobs:
  terraform:
    uses: v-grand/infra-ci/.github/workflows/terraform-plan.yml@main
    with:
      terraform_version: '1.5.0'
      working_directory: './'
    secrets:
      aws_access_key_id: ${{ secrets.AWS_ACCESS_KEY_ID }}
      aws_secret_access_key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}

  deploy_backend:
    needs: terraform
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Configure AWS credentials
        uses: aws-actions/configure-aws-credentials@v2
        with:
          aws-access-key-id: ${{ secrets.AWS_ACCESS_KEY_ID }}
          aws-secret-access-key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
          aws-region: us-east-1
      
      - name: Build and push Docker image
        run: |
          aws ecr get-login-password | docker login --username AWS --password-stdin $ECR_REGISTRY
          docker build -t my-webapp-backend ./backend
          docker tag my-webapp-backend:latest $ECR_REGISTRY/my-webapp-backend:latest
          docker push $ECR_REGISTRY/my-webapp-backend:latest
      
      - name: Deploy to ECS
        run: |
          aws ecs update-service --cluster my-webapp-cluster --service backend --force-new-deployment

  deploy_frontend:
    needs: terraform
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Setup Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '18'
      
      - name: Build frontend
        run: |
          cd frontend
          npm ci
          npm run build
      
      - name: Deploy to S3
        run: |
          aws s3 sync frontend/build/ s3://my-webapp-frontend --delete
          aws cloudfront create-invalidation --distribution-id $CLOUDFRONT_ID --paths "/*"

## Шаг 6: Настройка мониторинга (infra-monitoring)

Интегрируем Prometheus, Grafana и CloudWatch для мониторинга.

In [ ]:
%%writefile modules/monitoring/main.tf

# CloudWatch Alarms
resource "aws_cloudwatch_metric_alarm" "backend_cpu" {
  alarm_name          = "${var.project_name}-backend-cpu-high"
  comparison_operator = "GreaterThanThreshold"
  evaluation_periods  = "2"
  metric_name         = "CPUUtilization"
  namespace           = "AWS/ECS"
  period              = "300"
  statistic           = "Average"
  threshold           = "80"
  alarm_description   = "This metric monitors ECS CPU utilization"
  
  dimensions = {
    ClusterName = module.ecs_cluster.cluster_name
    ServiceName = "backend"
  }
}

resource "aws_cloudwatch_metric_alarm" "database_connections" {
  alarm_name          = "${var.project_name}-db-connections-high"
  comparison_operator = "GreaterThanThreshold"
  evaluation_periods  = "1"
  metric_name         = "DatabaseConnections"
  namespace           = "AWS/RDS"
  period              = "300"
  statistic           = "Average"
  threshold           = "80"
  
  dimensions = {
    DBInstanceIdentifier = module.database.db_instance_id
  }
}

# Grafana Dashboard (deployed on ECS)
module "grafana" {
  source = "git::https://github.com/v-grand/infra-monitoring.git//modules/grafana"

  vpc_id          = module.network.vpc_id
  subnet_ids      = module.network.private_subnets
  ecs_cluster_id  = module.ecs_cluster.cluster_id
}

## Шаг 7: Проверка развертывания

Проверяем, что все компоненты работают корректно.

In [ ]:
import boto3
import requests

# Проверка ECS сервисов
ecs = boto3.client('ecs', region_name='us-east-1')
services = ecs.describe_services(
    cluster='my-webapp-cluster',
    services=['backend']
)

print("Backend Service Status:")
for service in services['services']:
    print(f"  - {service['serviceName']}: {service['status']}")
    print(f"  - Running tasks: {service['runningCount']}")
    print(f"  - Desired tasks: {service['desiredCount']}")

# Проверка RDS
rds = boto3.client('rds', region_name='us-east-1')
db_instances = rds.describe_db_instances(
    DBInstanceIdentifier='my-webapp-db'
)

print("\nDatabase Status:")
for db in db_instances['DBInstances']:
    print(f"  - Status: {db['DBInstanceStatus']}")
    print(f"  - Endpoint: {db['Endpoint']['Address']}")

# Проверка Frontend (S3)
s3 = boto3.client('s3')
try:
    response = s3.head_bucket(Bucket='my-webapp-frontend')
    print("\nFrontend Bucket: ✅ Exists")
except:
    print("\nFrontend Bucket: ❌ Not found")

# Проверка доступности приложения
# alb_url = "http://my-webapp-alb-123456789.us-east-1.elb.amazonaws.com"
# response = requests.get(f"{alb_url}/health")
# print(f"\nApplication Health Check: {response.status_code}")

## Заключение

### Что мы развернули:

✅ **Сеть**: VPC с публичными и приватными подсетями, Tailscale для безопасного доступа  
✅ **База данных**: RDS PostgreSQL с автоматическими бэкапами  
✅ **Backend**: ECS Fargate с автоскейлингом  
✅ **Frontend**: S3 + CloudFront для статического контента  
✅ **Секреты**: SOPS для шифрования чувствительных данных  
✅ **CI/CD**: GitHub Actions для автоматического развертывания  
✅ **Мониторинг**: CloudWatch алармы и Grafana дашборды  

### Следующие шаги:

1. Настроить автоскейлинг для ECS сервисов
2. Добавить WAF для защиты от атак
3. Настроить Route53 для DNS
4. Добавить SSL сертификаты через ACM
5. Настроить логирование в CloudWatch Logs
6. Создать disaster recovery план

### Полезные команды:

```bash
# Просмотр логов ECS
aws logs tail /ecs/my-webapp-backend --follow

# Подключение к БД через Tailscale
psql -h <rds-endpoint> -U admin -d webapp_db

# Расшифровка секретов
sops -d secrets.enc.yaml

# Откат deployment
terraform plan -destroy
```